# Environment Demo

In [17]:
import numpy as np
import pygame

import gymnasium as gym
from gymnasium import spaces
from math import sqrt, atan2, degrees

## Robot features

- 2.5 cm diameter
- compass
- 360 vision sensor and object reconition in range 0.5m
- comunication between others robots (0.5m range)
- ability to pick up stuff (in they're in the same position of the object)
- holonomic motion (every directions)

In [18]:
ROBOT_SIZE = 2.5
SENSOR_RANGE = 50

Since we are in a 2d gridworld environment a robot can move only in a discrete space of directions. The robot can move in 8 directions (N, NE, E, SE, S, SW, W, NW). The robot can also pick up objects and put them down.

In [19]:
STAY = 0
N = 1
NE = 2
E = 3
SE = 4
S = 5
SW = 6
W = 7
NW = 8
PICK_UP = 9
PUT_DOWN = 10

The robots have 16 sensors, so it's safe to assume that they could detect the closest objects in all 360 angle. The sensor also allows the robot to detect the distance to the object and the type of the object. So we can define the sensor as a tuple (angle, distance, object_type) and each robot would have a list of 16 of these tuples.

## Arena

5m x 5m with robots and colored objects 

In [20]:
ARENA_SIZE = 500 / ROBOT_SIZE

## Environment construction

In [89]:
class GridWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}
    def __init__(
            self, 
            render_mode=None, 
            size=100, 
            n_agents=3, 
            n_blocks=3, 
            n_sensors = 6, 
            sensor_range=5,
            sensor_degree = 360,
            seed=None
            ):
        
        self.size = size  # The size of the square grid
        self.window_size = 512  # The size of the PyGame window

        self._n_agents = n_agents
        self.n_blocks = n_blocks

        self._n_sensors = n_sensors
        self._sensors_range = sensor_range
        self._sensors_degree = sensor_degree
        self._sensors_angle = self._sensors_degree / self._n_sensors

        self._agents_locations = np.zeros((self._n_agents, 2), dtype=int)
        
        self._sensors = np.zeros((self._n_agents, n_sensors, 3), dtype=int) # init sensors
        
        # All directions
        self._action_to_direction = {
            STAY : np.array([0, 0]),
            N : np.array([0, -1]),
            NE : np.array([1, -1]),
            E : np.array([1, 0]),
            SE : np.array([1, 1]),
            S : np.array([0, 1]),
            SW : np.array([-1, 1]),
            W : np.array([-1, 0]),
            NW : np.array([-1, -1]),
        }
        self._other_actions = [PICK_UP, PUT_DOWN]

        n_total_actions = len(self._action_to_direction) + len(self._other_actions)
        
        self.action_space = spaces.Tuple([spaces.Discrete(n_total_actions) for _ in range(self._n_agents)])

        self.observation_space = spaces.Dict(
            {
                "sensors": spaces.Box(0, 255, shape=(self._n_agents, n_sensors, 3), dtype=int),
            }
        )

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode
        self.window = None
        self.clock = None

    def _calculate_distance_direction(self, pointA, pointB, distance_type='euclidean'):
        """
        Calculate the distance and direction in degrees from pointA to pointB in a grid world.

        Parameters:
        - pointA: Tuple[int, int] representing the coordinates of the first point (x1, y1).
        - pointB: Tuple[int, int] representing the coordinates of the second point (x2, y2).
        - distance_type: String indicating the type of distance to calculate ('manhattan' or 'euclidean').

        Returns:
        - distance: The calculated distance between the two points.
        - direction_degrees: The direction from pointA to pointB in degrees from 0 to 360.
        """
        x1, y1 = pointA
        x2, y2 = pointB

        # Calculate distance
        if distance_type == 'manhattan':
            distance = abs(x1 - x2) + abs(y1 - y2)
        elif distance_type == 'euclidean':
            distance = sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
        else:
            raise ValueError("Invalid distance type. Use 'manhattan' or 'euclidean'.")

        # Calculate direction in degrees
        angle_radians = atan2(y2 - y1, x2 - x1)
        direction_degrees = degrees(angle_radians)
        # TODO: south is 0 degrees, east is 90 degrees, north is 180 degrees, west is -90 degrees


        return distance, direction_degrees
    
    def _get_obs(self):
        # Mimic sensors reading
        for i in range(self._n_agents):
            for j in range(self._n_agents):
                if i != j:

                    distance, direction = self._calculate_distance_direction(self._agents_locations[i], 
                                                                            self._agents_locations[j])
                    if distance <= self._sensors_range:
                        sensor_index = int(direction / self._sensors_angle)
                        self._sensors[i, sensor_index] = [direction, distance, j]
                        print(f"Agent {i} sees agent {j} at {direction} degrees and {distance} distance")
                    else:
                        print(f"Agent {i} doesn't see agent {j}")
                        self._sensors[i, :] = [0, 0, 0]
                
        
        return {"sensors": self._sensors}
                
    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        for i in range(self._n_agents):
            self._agents_locations[i] = self.np_random.integers(0, self.size, size=2, dtype=int)

        observation = self._get_obs()
        # info = self._get_info()
        info = None

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def step(self, action):
        for i in range(self._n_agents):
            
            # Map the action (element of {0,1,2,3}) to the direction we walk in
            direction = self._action_to_direction[action[i]]
            
            # We use `np.clip` to make sure we don't leave the grid
            self._agents_locations[i] = np.clip(
                self._agents_locations[i] + direction, 0, self.size - 1
            )
        # An episode is done iff the agent has reached the target
        # terminated = np.array_equal(self._agent_location, self._target_location)
        # reward = 1 if terminated else 0  # Binary sparse rewards
        observation = self._get_obs()

        if self.render_mode == "human":
            self._render_frame()

        return observation, None, None, False, None

    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()

    def _render_frame(self):
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (self.window_size, self.window_size)
            )
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
            self.window_size / self.size
        )  # The size of a single grid square in pixels

        # First we draw the target
        pygame.draw.rect(
            canvas,
            (255, 0, 0),
            pygame.Rect(
                pix_square_size * self._target_location,
                (pix_square_size, pix_square_size),
            ),
        )
        # Now we draw the agent
        pygame.draw.circle(
            canvas,
            (0, 0, 255),
            (self._agent_location + 0.5) * pix_square_size,
            pix_square_size / 3,
        )

        # Finally, add some gridlines
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        if self.render_mode == "human":
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
        
    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

In [90]:
env = GridWorldEnv(render_mode='rgb_array', size=5, n_agents=3, n_blocks=3)

In [91]:
initial_state = env.reset()

Agent 0 sees agent 1 at -116.56505117707799 degrees and 4.47213595499958 distance
Agent 0 sees agent 2 at -135.0 degrees and 2.8284271247461903 distance
Agent 1 sees agent 0 at 63.43494882292201 degrees and 4.47213595499958 distance
Agent 1 sees agent 2 at 90.0 degrees and 2.0 distance
Agent 2 sees agent 0 at 45.0 degrees and 2.8284271247461903 distance
Agent 2 sees agent 1 at -90.0 degrees and 2.0 distance


In [92]:
env._agents_locations

array([[4, 4],
       [2, 0],
       [2, 2]])

In [80]:
initial_state

({'sensors': array([[[ 26,   2,   2],
          [  0,   0,   0],
          [  0,   0,   0],
          [206,   2,   1],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[  0,   0,   0],
          [ 90,   2,   2],
          [153,   2,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          [270,   2,   1],
          [333,   2,   0]]])},
 None)

In [ ]:
initial_state = env.reset()
print(initial_state)
step = env.action_space.sample()
print(step)
next_state = env.step(step)
print(next_state)

({'locations': [array([ 0, 21]), array([49,  9]), array([49,  3])]}, None)
(1, 1, 3)
({'locations': [array([ 0, 22]), array([49, 10]), array([49,  2])]}, None, None, False, None)


In [105]:


def calculate_distance_direction_degrees(pointA, pointB, distance_type='euclidean'):
    """
    Calculate the distance and direction in degrees from pointA to pointB in a grid world.

    Parameters:
    - pointA: Tuple[int, int] representing the coordinates of the first point (x1, y1).
    - pointB: Tuple[int, int] representing the coordinates of the second point (x2, y2).
    - distance_type: String indicating the type of distance to calculate ('manhattan' or 'euclidean').

    Returns:
    - distance: The calculated distance between the two points.
    - direction_degrees: The direction from pointA to pointB in degrees from 0 to 360.
    """
    x1, y1 = pointA
    x2, y2 = pointB

    # Calculate distance
    if distance_type == 'manhattan':
        distance = abs(x1 - x2) + abs(y1 - y2)
    elif distance_type == 'euclidean':
        distance = sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    else:
        raise ValueError("Invalid distance type. Use 'manhattan' or 'euclidean'.")

    # Calculate direction in degrees
    angle_radians = atan2(y2 - y1, x2 - x1)
    direction_degrees = degrees(angle_radians)
    # direction_degrees = (90 - direction_degrees) % 360 
    
    return distance, direction_degrees

In [107]:
# Example usage
pointA = (3, 3)
pointB = (0, 3)

distance, direction_degrees = calculate_distance_direction_degrees(pointA, pointB, 'manhattan')
print(f"Manhattan distance: {distance}, Direction: {direction_degrees} degrees")

distance, direction_degrees = calculate_distance_direction_degrees(pointA, pointB, 'euclidean')
print(f"Euclidean distance: {distance}, Direction: {direction_degrees} degrees")

Manhattan distance: 3, Direction: 270.0 degrees
Euclidean distance: 3.0, Direction: 270.0 degrees
